In [ ]:
#!pip install jcopdl
#!pip install gdown 

In [ ]:
# Download Datasets
!gdown https://drive.google.com/uc?id=12DT5Px7FQV7gEcyGWvKb5aZQW2ZptSP
!unzip /content/mnist.zip

In [1]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Datasets dan Dataloader (Hanya Trainset)

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from jcopdl.layers import linear_block
from torch import nn


In [ ]:
bs = 64

data_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # normalize supaya menjadi -1 sampai 1, supaya lebih stabil
])

train_set = datasets.ImageFolder("data/train/", transform=data_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=8)

## Arsitektur dan Config

In [5]:
"""import torch
from torch import nn
from jcopdl.layers import linear_block

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            linear_block(784, 512, activation="lrelu"),
            linear_block(512, 256, activation="lrelu"),
            linear_block(256, 128, activation="lrelu"),
            linear_block(128, 1, activation='sigmoid')
        )
    
    def forward(self, x):
        return self.fc(x)
    

class Generator(nn.Module):
    def __init__(self, z_dim):
        super().__init__()
        self.z_dim = z_dim
        self.fc = nn.Sequential(
            linear_block(z_dim, 128, activation="lrelu"),
            linear_block(126, 256, activation="lrelu", batch_norm=True),
            linear_block(256, 512, activation="lrelu", batch_norm=True),
            linear_block(512, 1024, activation="lrelu", batch_norm=True),
            linear_block(1024, 784, activation="tanh")
        )
    
    def forward(self, x):
        return self.fc(x)
    
    def generate(self, n, device):
        z = torch.randn((n, self.z_dim), device=device)
        return self.fc(z)"""

'import torch\nfrom torch import nn\nfrom jcopdl.layers import linear_block\n\nclass Discriminator(nn.Module):\n    def __init__(self):\n        super().__init__()\n        self.fc = nn.Sequential(\n            nn.Flatten(),\n            linear_block(784, 512, activation="lrelu"),\n            linear_block(512, 256, activation="lrelu"),\n            linear_block(256, 128, activation="lrelu"),\n            linear_block(128, 1, activation=\'sigmoid\')\n        )\n    \n    def forward(self, x):\n        return self.fc(x)\n    \n\nclass Generator(nn.Module):\n    def __init__(self, z_dim):\n        super().__init__()\n        self.z_dim = z_dim\n        self.fc = nn.Sequential(\n            linear_block(z_dim, 128, activation="lrelu"),\n            linear_block(126, 256, activation="lrelu", batch_norm=True),\n            linear_block(256, 512, activation="lrelu", batch_norm=True),\n            linear_block(512, 1024, activation="lrelu", batch_norm=True),\n            linear_block(1024, 78

In [6]:
config = set_config({
    "z_dim" : 100,
    "bs" :64
})

## training Preparation

In [7]:
from model_gan import Discriminator, Generator

In [8]:
D_model = Discriminator().to(device)
G_model = Generator(config.z_dim).to(device)

criterion = nn.BCELoss()

d_optimizer = optim.Adam(D_model.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G_model.parameters(), lr=0.0002)

# Fase Training

In [11]:
import os

from torchvision.utils import save_image

In [10]:
os.makedirs("output/", exist_ok=True)
os.makedirs("model/", exist_ok=True)

In [ ]:
num_epochs = 300

for epoch in range(num_epochs):
    D_model.train()
    G_model.train()
    
    for real_image, _ in trainloader:
        jumlah_data = real_image.shape[0]
        
        # Handle Real image dan Fake image
        real_image = real_image.to(device)
        fake_image = G_model.generate(jumlah_data, device=device)
        
        # Handle Real label dan Fake label
        real_label = torch.ones((jumlah_data, 1), device=device)
        fake_label = torch.zeros((jumlah_data, 1), device=device)
        
        # Training Discriminator
        d_optimizer.zero_grad()
        ## 1. Real image -> Discriminator -> label real
        output = D_model(real_image)
        d_real_loss = criterion(output, real_label)
        
        ## 2. Fake image -> Discriminator -> label fake
        output = D_model(fake_image.detach())
        d_fake_loss = criterion(output, fake_label)
        
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()
        
        # Training Generator
        g_optimizer.zero_grad()
        
        
    if epoch % 5 == 0:
        print(f"Epoch: {epoch:5} | D_loss: {d_loss/2:.5f} | G_loss: {g_loss/2:.5f}")
        
    if epoch % 20 == 0:
        G_model.eval()
        epoch = str(epoch).zfill(3)
        fake_image = G_model.generate(64, device=device)
        save_image(fake_image.view(-1, 1, 28, 28), f"output/{epoch}.jpg", nrow=8, normalize=True)
        
        #save model
        torch.save(D_model, "model/discriminator.pth")
        torch.save(G_model, "model/generator.pth")